## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from pprint import pprint
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from IPython.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 10
NUMBER_OF_ACTIVATED_PARTICIPANTS = 5
NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [20_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True
PREFIX = 'terra'

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(43)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefix: str = PREFIX) -> dict:

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 10
Number of threads: 100


  0%|          | 0/10 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,20000,john16504256240,QmP4nToEBpi6uiWj5Nb2au4dR17tkYy15fXgXqEYGBusJm,0x9a5f0bbf5c11d6a529cd8117554a8081027846fb,0xfe86a12db9a254e9d218ad1ea5fea014689a02e3b8f6...,bostrom1mpxadqzwgfahat2u8ch5lf398sw3gyggtqlww6,terra1mpxadqzwgfahat2u8ch5lf398sw3gyggwh3aja,card choose elevator truly lottery write denia...,bostrom1mpxadqzwgfahat2u8ch5lf398sw3gyggtqlww6...,0xd5cd520a249cfb60ccea30795d65765e652525f2e5db...,{'pub_key': 'As9N9jZB+OnV7elydUz7l0sHCp1g0Qyn/...,eyJwdWJfa2V5IjoiQXM5TjlqWkIrT25WN2VseWRVejdsMH...
1,20000,john16504256241,Qme38z9BvkBasFUNrhn8TVvrWD8ePpGkpNx8v7WJWvyTHt,0xafd2bdcfb28b845c92f4ece5a6c44fdbceba1b61,0xf184592e269f930ed1211e5953c4c5703fbf283c05a1...,bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq,terra144tp5hje0eflst8eg8py8jg0r4lhfyjhx8ghq8,ability hour sleep stem jewel industry respons...,bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq...,0x95bb7ad4e0ff4e8ff1a95515d9eaf0467aef73abe253...,{'pub_key': 'At1Gc8lVuvPmiY3cBO3n2NBIUK/qak6fy...,eyJwdWJfa2V5IjoiQXQxR2M4bFZ1dlBtaVkzY0JPM24yTk...
2,20000,john16504256242,QmVmT11r27GhwMitNCiP6WM2nRDsizZZazRRFBD4QBwWre,0x905a168d90cff99902baf6297f682a0a1727cef2,0xc36d13f58573be843be6d38ef21ed71bebf6fc37ef3a...,bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8,terra1dzkua4flhf5g5vkzc37ne3dwzwf2pry7elcvtq,fog screen cluster vivid exhibit violin myth e...,bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8...,0xd7df657766b823f958f2ac687c13cdf47aab36546e1f...,{'pub_key': 'AjXCkZZoFKOjTXZ5B1RHjqE9ntRHRSsED...,eyJwdWJfa2V5IjoiQWpYQ2taWm9GS09qVFhaNUIxUkhqcU...
3,20000,john16504256243,QmPyJz5gJmVZkNHnjLtssqufLwQaQw7FPDgD7CY5uHfHNc,0xb70ed2815618997c30da21244583e6c6401af6cb,0x0d1d64c44de49655fcfeafd45194cc2cfdca0d652fb7...,bostrom1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gfq6c6k,terra1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gvh5tx3,huge arch muscle wrestle pig party mouse stree...,bostrom1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gfq6c6k...,0x09aef9028c2ac4737068427a2ce96ff2b82718d494f4...,{'pub_key': 'AlIlP/46XgHznglIjjoOOy9IAKv00CHdM...,eyJwdWJfa2V5IjoiQWxJbFAvNDZYZ0h6bmdsSWpqb09PeT...
4,20000,john16504256244,QmT2u7k3buV7Nhii1NoyoJNp8j1PskgzcynisqPX5vhGFR,0x88688c62f9343cbc8135046f38d64c905e11c5d9,0x4c9e62a47f27b72a2f287f8ca80b22dbc4d66bd406a1...,bostrom1ng4677w8sq48dsvhxpqqrs07enqn43v5fhtt7s,terra1ng4677w8sq48dsvhxpqqrs07enqn43v5vq9czh,lab tone caught reunion urban another loyal sk...,bostrom1ng4677w8sq48dsvhxpqqrs07enqn43v5fhtt7s...,0x3ca4073a15d383f44bf852294bbb155d689dbcdbde02...,{'pub_key': 'A8aj971kLRSLraxvGk2CO6x7T0CibPf5R...,eyJwdWJfa2V5IjoiQThhajk3MWtMUlNMcmF4dkdrMkNPNn...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item[0],
                     'amount': str(_item[1])} for _item in claims_df[['ethereum_address', 'amount']].values.tolist()]+ \
                    [{'address': _item[0],
                      'amount': str(_item[1])} for _item in claims_df[['cosmos_address', 'amount']].values.tolist()]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 15
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(tasks)
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

{('root_testing_source_10_addresses.json', 'temp/proofs_0.json', 0, 21)}


  0%|          | 0/1 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_10_addresses.json --output temp/proofs_0.json --start_index 0 --end_index 21
Merkle root: 38972df4bb090bf59f5f762d03683fe741f742ae66b0dc96906d02cc8b0be033
Number of addresses in the Merkle tree: 21
Done in 1.14s.



In [5]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[proofs_df.address.str.startswith(PREFIX)][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='left',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='left',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,20000,john16504256240,QmP4nToEBpi6uiWj5Nb2au4dR17tkYy15fXgXqEYGBusJm,0x9a5f0bbf5c11d6a529cd8117554a8081027846fb,0xfe86a12db9a254e9d218ad1ea5fea014689a02e3b8f6...,bostrom1mpxadqzwgfahat2u8ch5lf398sw3gyggtqlww6,terra1mpxadqzwgfahat2u8ch5lf398sw3gyggwh3aja,card choose elevator truly lottery write denia...,bostrom1mpxadqzwgfahat2u8ch5lf398sw3gyggtqlww6...,0xd5cd520a249cfb60ccea30795d65765e652525f2e5db...,{'pub_key': 'As9N9jZB+OnV7elydUz7l0sHCp1g0Qyn/...,eyJwdWJfa2V5IjoiQXM5TjlqWkIrT25WN2VseWRVejdsMH...,[c11b8fd1bf4acbb10d9a2938934798692f31eca0c5e98...,[e30b13867a702d325c698dbf43db1229fb531d43d32e6...
1,20000,john16504256241,Qme38z9BvkBasFUNrhn8TVvrWD8ePpGkpNx8v7WJWvyTHt,0xafd2bdcfb28b845c92f4ece5a6c44fdbceba1b61,0xf184592e269f930ed1211e5953c4c5703fbf283c05a1...,bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq,terra144tp5hje0eflst8eg8py8jg0r4lhfyjhx8ghq8,ability hour sleep stem jewel industry respons...,bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq...,0x95bb7ad4e0ff4e8ff1a95515d9eaf0467aef73abe253...,{'pub_key': 'At1Gc8lVuvPmiY3cBO3n2NBIUK/qak6fy...,eyJwdWJfa2V5IjoiQXQxR2M4bFZ1dlBtaVkzY0JPM24yTk...,[a76348d8a6006c0d04fb046ebe968c80a9371ae1dc4a4...,[eb00b5aac8ff0113e4c011c2920ea161d10dab7cf48b5...
2,20000,john16504256242,QmVmT11r27GhwMitNCiP6WM2nRDsizZZazRRFBD4QBwWre,0x905a168d90cff99902baf6297f682a0a1727cef2,0xc36d13f58573be843be6d38ef21ed71bebf6fc37ef3a...,bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8,terra1dzkua4flhf5g5vkzc37ne3dwzwf2pry7elcvtq,fog screen cluster vivid exhibit violin myth e...,bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8...,0xd7df657766b823f958f2ac687c13cdf47aab36546e1f...,{'pub_key': 'AjXCkZZoFKOjTXZ5B1RHjqE9ntRHRSsED...,eyJwdWJfa2V5IjoiQWpYQ2taWm9GS09qVFhaNUIxUkhqcU...,[39e347286602d0d2ccb3ed52293967d23735e7648b4e9...,[8a28c78c36b5cd5f35dd0294a7d9ac3b27bea0cb8d23d...
3,20000,john16504256243,QmPyJz5gJmVZkNHnjLtssqufLwQaQw7FPDgD7CY5uHfHNc,0xb70ed2815618997c30da21244583e6c6401af6cb,0x0d1d64c44de49655fcfeafd45194cc2cfdca0d652fb7...,bostrom1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gfq6c6k,terra1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gvh5tx3,huge arch muscle wrestle pig party mouse stree...,bostrom1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gfq6c6k...,0x09aef9028c2ac4737068427a2ce96ff2b82718d494f4...,{'pub_key': 'AlIlP/46XgHznglIjjoOOy9IAKv00CHdM...,eyJwdWJfa2V5IjoiQWxJbFAvNDZYZ0h6bmdsSWpqb09PeT...,[17c7ae3fd12ce47596f08f97dfae61f0f8bc62d3e37bd...,[67c905493fb16671a33a7da378475269a41b66cdd4e14...
4,20000,john16504256244,QmT2u7k3buV7Nhii1NoyoJNp8j1PskgzcynisqPX5vhGFR,0x88688c62f9343cbc8135046f38d64c905e11c5d9,0x4c9e62a47f27b72a2f287f8ca80b22dbc4d66bd406a1...,bostrom1ng4677w8sq48dsvhxpqqrs07enqn43v5fhtt7s,terra1ng4677w8sq48dsvhxpqqrs07enqn43v5vq9czh,lab tone caught reunion urban another loyal sk...,bostrom1ng4677w8sq48dsvhxpqqrs07enqn43v5fhtt7s...,0x3ca4073a15d383f44bf852294bbb155d689dbcdbde02...,{'pub_key': 'A8aj971kLRSLraxvGk2CO6x7T0CibPf5R...,eyJwdWJfa2V5IjoiQThhajk3MWtMUlNMcmF4dkdrMkNPNn...,[837f5204efbf16ba6ef9cf0680010f79876a313e9c811...,[a484f51b4afcea3792bf7f3f7864ddf2ec94774c243f5...


In [6]:
for i in range(ceil(NUMBER_OF_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [10]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1d53n4k00ywxu9al2g8a8wnjvp8cxdeu9w7vve96vw4raasf9arqqtp6xpj


### Initiate Class of Output Parsing

In [11]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [12]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 38972df4bb090bf59f5f762d03683fe741f742ae66b0dc96906d02cc8b0be033
Gas used: 121,492
Tx hash: 18BD6190A6CF7E929E2EE8777C4845479B6F94070CEFB5A81927F96259DF56A2


#### Get Merkle Root form the Gift Contract

In [13]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1d53n4k00ywxu9al2g8a8wnjvp8cxdeu9w7vve96vw4raasf9arqqtp6xpj
{'data': {'merkle_root': '38972df4bb090bf59f5f762d03683fe741f742ae66b0dc96906d02cc8b0be033'}}


### Send coins to new addresses

In [14]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 750

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in tqdm(bostrom_addresses_chunks):
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=bostrom_addresses_item,
            amounts=[1] * len(bostrom_addresses_item),
            gas=min(100_000 + 25_000 * len(bostrom_addresses_item), int(24e6)),
            display_data=DISPLAY_TX_EXECUTION)
    contract_utils.parse_contract_execution_json(contract_execution_json=send_output)

  0%|          | 0/1 [00:00<?, ?it/s]


Events

coin received
	receiver: bostrom1mpxadqzwgfahat2u8ch5lf398sw3gyggtqlww6
	amount: 1boot
	receiver: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	amount: 1boot
	receiver: bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8
	amount: 1boot
	receiver: bostrom1pnnyyhz95r3ne0vqpwx5m7d0g7xvwr6gfq6c6k
	amount: 1boot
	receiver: bostrom1ng4677w8sq48dsvhxpqqrs07enqn43v5fhtt7s
	amount: 1boot
	receiver: bostrom1rd69gcc3jyfj2tkev975aehdgc5vc39wgpf733
	amount: 1boot
	receiver: bostrom1uu2d4eujf9vvhjak7texgnxumdrzyxzzvsd3v4
	amount: 1boot
	receiver: bostrom10ymyd7z635s3v94n93tdc5084mfhxsyg6v42um
	amount: 1boot
	receiver: bostrom18v75rmhlpj2pkux4g0yghry656dr2qr8qszjvz
	amount: 1boot
	receiver: bostrom17nxuf2fsv09esy4p97zgnxkt4fsltyznfycrz8
	amount: 1boot

coin spent
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1

### Create Passports, Proof Addresses and Claim Gift

In [15]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 5
Number of threads: 100


  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
with open('temp/contract_participation_execution_log_1.txt', 'r') as f:
    data = json.load(f)

In [17]:
pprint(data)

{'claim_cosmos': '{"code":0,"codespace":"","data":null,"gas_used":314457,"gas_wanted":400000,"height":2297173,"info":null,"logs":[{"events":[{"attributes":[{"key":"receiver","value":"bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq"},{"key":"amount","value":"100000boot"}],"type":"coin_received"},{"attributes":[{"key":"spender","value":"bostrom1d53n4k00ywxu9al2g8a8wnjvp8cxdeu9w7vve96vw4raasf9arqqtp6xpj"},{"key":"amount","value":"100000boot"}],"type":"coin_spent"},{"attributes":[{"key":"_contract_address","value":"bostrom1d53n4k00ywxu9al2g8a8wnjvp8cxdeu9w7vve96vw4raasf9arqqtp6xpj"}],"type":"execute"},{"attributes":[{"key":"action","value":"/cosmwasm.wasm.v1.MsgExecuteContract"},{"key":"module","value":"wasm"},{"key":"sender","value":"bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq"}],"type":"message"},{"attributes":[{"key":"recipient","value":"bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq"},{"key":"sender","value":"bostrom1d53n4k00ywxu9al2g8a8wnjvp8cxdeu9w7vve96vw4raasf9arqqtp6xpj"},{"key"

In [18]:
contract_utils.parse_contract_execution_json(data['create'])


Events

cyberlinks
	QmNwfH86ac37xhBsKh4nHHAanv8jJeCRG5VEZG83EHvnQv -> QmXz6t1pY4oxu5gmX5wTc9URmmcJyokqT3LcBqr92aSbpY
	QmXz6t1pY4oxu5gmX5wTc9URmmcJyokqT3LcBqr92aSbpY -> QmNwfH86ac37xhBsKh4nHHAanv8jJeCRG5VEZG83EHvnQv
	neuron: Name Subgraph Contract

	QmXz6t1pY4oxu5gmX5wTc9URmmcJyokqT3LcBqr92aSbpY -> Qme38z9BvkBasFUNrhn8TVvrWD8ePpGkpNx8v7WJWvyTHt
	Qme38z9BvkBasFUNrhn8TVvrWD8ePpGkpNx8v7WJWvyTHt -> QmXz6t1pY4oxu5gmX5wTc9URmmcJyokqT3LcBqr92aSbpY
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 273396
Gas used: 440,639
Tx hash: AEB78D27926A2870BF0290A90D3CA7B949196FE2521119BA16117A5C6D105D5D


In [19]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'])


Events

cyberlinks
	QmXz6t1pY4oxu5gmX5wTc9URmmcJyokqT3LcBqr92aSbpY -> QmTFdFAmuRSK5JvYZT9Tphf5CBzDPtWUNJP5NgEuvKP1DZ
	QmTFdFAmuRSK5JvYZT9Tphf5CBzDPtWUNJP5NgEuvKP1DZ -> QmXz6t1pY4oxu5gmX5wTc9URmmcJyokqT3LcBqr92aSbpY
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16504256241
	address: terra144tp5hje0eflst8eg8py8jg0r4lhfyjhx8ghq8
Gas used: 284,668
Tx hash: 282376D133C956370A4F3F6F658A3724BBD2F7ADAEB113580B7E1708A8B6DBEF


In [20]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'])


Events

coin received
	receiver: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: terra144tp5hje0eflst8eg8py8jg0r4lhfyjhx8ghq8
	target: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	amount: 259,781
Gas used: 314,457
Tx hash: 664139380EB6BA055A507BDAF9C645EE1AE08E7A2DFA3442A2CB7AB8FBD18297


In [21]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 5
Number of threads: 100


  0%|          | 0/5 [00:00<?, ?it/s]

In [22]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [23]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'])


Events

coin received
	receiver: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	amount: 15990boot

coin spent
	spender: Gift Contract
	amount: 15990boot

execute
	execute contract: Gift Contract

message from bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	sender: Gift Contract
	amount: 15990boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	gift_address: 0xafd2bdcfb28b845c92f4ece5a6c44fdbceba1b61
	stage: 9
	amount: 15,990
Gas used: 157,447
Tx hash: 5D59183EE4403DB7425BA8A87BD93AF1BD59CB2FC81931FA542813B5C0326ACA


## Passport NFT testing
### Transfer Passport

In [24]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [25]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom144tp5hje0eflst8eg8py8jg0r4lhfyjhrsxyuq
	recipient: Passport Owner Address
	token_id: 273396
Gas used: 386,223
Tx hash: BD20CBEF172D95F75E69EE7AB14AC3C36287B4E25AD9B1B1EC79776A34115A40


### Burn

In [26]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8 wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1dzkua4flhf5g5vkzc37ne3dwzwf2pry7ugklh8
	token_id: 273395
Gas used: 382,058
Tx hash: B5B341737C1C79EC63923DAC82F2EC5538ED2E1456C27B396CFDBE87065FF0DB


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
